# N-PUZZLE GREEDY LAYER SOLUTION

In [198]:
import heapq
import json
from collections import namedtuple
import random
from tqdm.auto import tqdm
import numpy as np 

In [199]:
PUZZLE_DIM = 4
RANDOMIZE_STEPS = 100_000 * (PUZZLE_DIM // 3)
FINAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

action = namedtuple('Action', ['pos1', 'pos2'])

## Helper functions

In [200]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    ''' Change the current state applying one action'''
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_final_sol(state: np.ndarray) -> bool:
    '''Check the final state'''
    return np.array_equal(state, FINAL_STATE)

In [201]:
def to_tuple(state: np.ndarray) -> tuple:
        return tuple(state.flatten())

## Random Initialization

Reproducible Initialization (Not used in this case to compare the result with the layer solution)

In [202]:
random.seed(RANDOMIZE_STEPS)

curr_state = FINAL_STATE
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    curr_state = do_action(curr_state, random.choice(available_actions(curr_state)))
    
print("Initial state: ")    
curr_state

Randomizing: 100%|██████████| 100000/100000 [00:01<00:00, 78826.08it/s]

Initial state: 


array([[14, 11, 13,  9],
       [15,  5,  7,  0],
       [10,  4,  2,  6],
       [12,  8,  1,  3]])

Initialization Taken from layer solver in order to compare the two methods

In [203]:
if PUZZLE_DIM == 3:
    curr_state = np.array([
        [5, 4, 6],
        [8, 7, 2],
        [0, 1, 3]])
elif PUZZLE_DIM == 4:
    curr_state = np.array([
        [ 8,  4,  0,  5],
        [ 9, 13, 15,  7],
        [10, 11,  6, 12],
        [ 1,  2,  3, 14]])

## Distance Functions

In [204]:
def manhattan_distance(state: np.ndarray) -> int:
    '''Return The manattan distance between current position and final position'''
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # Ignora il vuoto
                target_x, target_y = np.where(FINAL_STATE==value)
                distance += abs(x - target_x) + abs(y - target_y)
    return distance

## A-Star solver

#### Solve with A-Star Algorithm

In [205]:
def a_star_solver(initial_state: np.ndarray) -> list[np.ndarray]:
    """Trova la soluzione del puzzle usando A*."""
    open_set = []
    initial_tuple = to_tuple(initial_state)
    final_tuple = to_tuple(FINAL_STATE)
    heapq.heappush(open_set, (0, initial_tuple))
    came_from = {}
    g_score = {initial_tuple: 0}
    f_score = {initial_tuple: manhattan_distance(initial_state)}

    visited = set()

    while open_set:
        _, current_tuple = heapq.heappop(open_set)

        if current_tuple in visited:
            continue
        visited.add(current_tuple)

        # Convert tuple back to NumPy array
        current = np.array(current_tuple).reshape(initial_state.shape)

        # Check if we've found the solution
        if current_tuple == final_tuple:
            # Reconstruct the path
            path = []
            while current_tuple in came_from:
                path.append(np.array(current_tuple).reshape(initial_state.shape))
                current_tuple = came_from[current_tuple]
            path.append(initial_state)
            return path[::-1]

        # Expand neighbors
        for action in available_actions(current):
            neighbor = do_action(current, action)
            neighbor_tuple = to_tuple(neighbor)

            if neighbor_tuple in visited:
                continue

            tentative_g_score = g_score[current_tuple] + 1

            if neighbor_tuple not in g_score or tentative_g_score < g_score[neighbor_tuple]:
                came_from[neighbor_tuple] = current_tuple
                g_score[neighbor_tuple] = tentative_g_score
                f_score[neighbor_tuple] = tentative_g_score + manhattan_distance(neighbor)
                heapq.heappush(open_set, (f_score[neighbor_tuple], neighbor_tuple))

    return None

In [206]:
path = a_star_solver(curr_state)

print(f"Total moves: {len(path)}")
print("Final State:")
print(path[-1])

Total moves: 55
Final State:
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15  0]]
